<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/08_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmi Ensemble

<br>

![mush](https://s3-eu-central-1.amazonaws.com/hermanns-website-2/wp-content/uploads/2018/11/09142320/florian-van-duyn-383221-unsplash_resized.jpg)

<br>

[Image Credits](https://www.hermanns.com/why-mushrooms-talk-to-us/)

<br>

Fino ad ora abbbiamo visto quelli che vengono detti *modelli deboli* o *weak learner*.

Cosa vuol dire in sostanza. Se noi costruiamo un albero di classificazione quello è solo uno dei diversi modi che esistono per classificare quelle osservazioni. I classificatori deboli sono anche inclini ad andare in overfitting proprio perchè la loro struttura è una delle molteplici che si potrebbero generare.

Una soluzione a questo problema sono i modelli ***Ensemble*** che mettono insieme più modelli deboli per generare un modello robusto al rumore. Naturalmente i costi in terminin di risorse e tempi salgono.

## Dataset

Il dataset che useremo si chiama Mushrooms e classifica i funghi in velenosi e non velenosi. Potete trovare maggiorni informazioni sulla[ pagina dedicata dell'UCI ML.](http://archive.ics.uci.edu/ml/datasets/Mushroom)

Il dataset ha oltre 8000 osservazioni e 23 variabili tutte categoriche. Faremo un minimo di conversione sulle variabili ma nessuna analisi dei dati o preprocessing.

Il dataset e il codice sono presi dal [kaggle kernel a questo link](https://www.kaggle.com/drvader/random-forests-and-gradient-boosting/data).

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1bo0cQb7hXjx1HRK6wj1G9kkt5ErVP9bN'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('mushrooms.csv')

In [0]:
df = pd.read_csv("mushrooms.csv")
df.head()

# Preparazione del dataset

Costruiamo la variabile di risposta binaria:
* 0 non velenoso
* 1 velenoso

Applichiamo un OneHot Encoding su tutte le altre variabili.

In [0]:
df["class"].replace(["e", "p"], [1, 0], inplace= True)
y = df["class"]
df.drop("class", axis = 1, inplace = True)

In [0]:
X = pd.get_dummies(df)
X.head()

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state= 42)
print("Osservazioni train: ", len(X_train), "\n", "Osservazioni Test: ", len(X_test))

## Models

**N.B. Per questioni di tempi non faremo tuning dei parametri ma potete trovare riferimenti interessanti tra i link in fondo al notebook.**


### Random Forest

<br>

![RandomForest](https://miro.medium.com/max/2000/1*_B5HX2whbTs3DS8M6YBD_w@2x.png)

<br>

[Image Credits](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)

<br>

Il **Random Forest** è l'algoritmo che ha sicuramente il nome più affascinante di tutti.

Usa una tecnica chiamata *bagging* per allenare in maniera parallela N decision tree e successivamente con un sistema di pesi (banalmente il più frequente) sceglie la classificazione finale. Dall'allenamento di N decision tree deriva il nome di questo algoritmo.

#### Bagging

Il **Bagging** (o Bootstrap aggregation) è una tecnica che si basa sul bootstrap, ovvero un ricampionamento in cui da una serie di dati vengono estratti N campioni indipendenti e ogni campione viene dato come input ad uno degli N alberi.

<br>

![bootstrap](https://miro.medium.com/max/1400/1*lWnm3eJVe3uo95OcSg5jUA@2x.png)

<br>

[Image Credits](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)

<br>

In questo modo ogni albero protrebbe essere differente e il risultato molto più resistente all'overfitting.

Inoltre allenando più alberi in maniera indipendente vi è una selezione delle features più impattanti.

In [0]:
from sklearn.ensemble import RandomForestClassifier

rfcl = RandomForestClassifier(n_estimators = 20, random_state = 42)

rfcl.fit(X_train, y_train)

yhat_rfcl = rfcl.predict(X_test)

### Gradient Boosting

<br>

![boosting](https://miro.medium.com/max/2000/1*VGSoqefx3Rz5Pws6qpLwOQ@2x.png)

<br>

[Image Credits](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)

<br>

#### Boosting

A differenza del bagging il **Boosting** è un processo iterativo che migliora la stima ad ogni passo fino ad arrivare ad un modello finale. Ad esempio vediamo questi due passi:

**STEP 1**:

* abbiamo una serie di dati
* proviamo a fare fitting con una funziona logaritmica

<br>

![step1](https://iaml.it/blog/gradient-boosting/images/P4SrKe6.png)

<br>

[Image Credits](https://iaml.it/blog/gradient-boosting)

<br>

**STEP 2**:

* per un miglior fitting aggiungiamo una funzione seno che cattura gli andamenti oscillatori

<br>

![step2](https://iaml.it/blog/gradient-boosting/images/7EUfGSi.png)

<br>

[Image Credits](https://iaml.it/blog/gradient-boosting)

<br>

Questo è un esempio di come funzionano gli algortimi di boosting, un processo iterativo dove ad ogni passo si cerca di migliorare il fitting.

Esistono diversi algoritmi di boosting, i più utilizzati e noti sono:
* AdaBoost
* GradientBoost (da cui deriva [XGBoost](https://en.wikipedia.org/wiki/XGBoost) che è molto prababilmente l'algoritmo più vincente su Kaggle).

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbcl = GradientBoostingClassifier(n_estimators = 20, random_state = 42)

gbcl.fit(X_train, y_train)

yhat_gbcl = rfcl.predict(X_test)

### Valutazione delle performance

#### Curva di ROC

La curva di ROC è uno strumento per valutare i classificatori binari e consite, in termini semplici, nel rapporto tra allarmi veri e falsi allarmi.

Misura la porzione dei veri positivi trovati rispetto a quella dei falsi postitivi, un valore pari a 1 corrisponde ad una perfetta classificazione.

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score, plot_roc_curve

#### Random Forest

In [0]:
print(confusion_matrix(y_test, yhat_rfcl))

print("\n")

print(accuracy_score(y_test, yhat_rfcl))

In [0]:
plot_roc_curve(rfcl, X_test, y_test)

#### Gradient Boosting

In [0]:
print(confusion_matrix(y_test, yhat_gbcl))

print("\n")

print(accuracy_score(y_test, yhat_gbcl))

In [0]:
plot_roc_curve(gbcl, X_test, y_test)

## Link Utili

[Kaggle kernel](https://www.kaggle.com/drvader/random-forests-and-gradient-boosting/data)

[Gradient Boosting - IAML](https://iaml.it/blog/gradient-boosting)

[Ensemble methods: bagging, boosting and stacking](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)

[Gradient Boosting with Scikit-Learn, XGBoost, LightGBM, and CatBoost](https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/)

[Present The Feature Importance of A Random Forest Classifier](https://towardsdatascience.com/present-the-feature-importance-of-the-random-forest-classifier-99bb042be4cc)

[Hyperparameters Optimization for LightGBM, CatBoost and XGBoost Regressors using Bayesian Optimization.](https://medium.com/analytics-vidhya/hyperparameters-optimization-for-lightgbm-catboost-and-xgboost-regressors-using-bayesian-6e7c495947a9)
